In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
df = pd.read_excel("amz.xlsx")
df=df.drop(["Unnamed: 0.1","Unnamed: 0"], axis=1)

In [3]:
dataDict=df.to_dict("records")
for i in range(len(dataDict)):
    dataDict[i]['S.No.'] = i + 1

In [4]:
valid_data=[]
for i in dataDict:
    if len(i["url"].split("/")[-1]) == 10:
        valid_data.append(i)

In [5]:
def get_data(url):
    proxy="45.152.188.246:3128"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'TE': 'trailers',
    }

    response = requests.get(url, headers=headers,proxies={"http":proxy,"https":proxy},timeout=10)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html5lib')
        product_data = {}
        try:
            product_data["Product Title"] = soup.find('h1', attrs={"id":"title"}).text.strip()
        except:
            pass
        try:
            product_data["Product Image URL"] = soup.find("div", attrs={"id":"img-canvas"}).find("img").get("src")
        except:
            try:
                product_data["Product Image URL"] = soup.find("div",id="imgTagWrapperId").find("img").get("src")
            except:
                pass
        try:
            product_data["Price of the Product"] = soup.find("span", class_ = "a-size-base a-color-price a-color-price").text.strip().replace("\xa0","").replace(",",".")
        except:
            try:
                product_data["Price of the Product"] = soup.find("span", class_ = "a-button a-button-selected a-spacing-mini a-button-toggle format").find("span", class_ = "a-color-base").text.strip().replace("\xa0","").replace("ab ","").replace(",",".")
            except:
                try:
                    product_data["Price of the Product"] = soup.find("span", class_ = "a-section a-spacing-none aok-align-center").find("span",class_="a-price-whole").text
                except:
                    pass
        try:
            new_key=[]
            for i in soup.find("div", id = "detailBullets_feature_div").find_all("li"):
                for j in i.find_all('span', class_ = "a-text-bold"):
                    new_key.append(j.text.replace("\u200f\n","").replace("\n","").replace("\u200e","").strip())
            product_key = []
            for i in new_key:
                product_key.append(i.split(":")[0].strip())
            value = []
            for i in soup.find("div", id = "detailBullets_feature_div").find_all("li"):
                value.append(i.span.text.split(":")[-1].replace(" \u200e\n","").replace("\n","").strip())
            product_data.update(dict(zip(product_key,value)))
        except:
            try:
                product_details_key = []
                for i in soup.find("table", id = "productDetails_techSpec_section_1").find_all("th"):
                    product_details_key.append(i.text)
                product_details_value = []
                for j in soup.find("table", id = "productDetails_techSpec_section_1").find_all("td"):
                    product_details_value.append(j.text.replace("\n","").strip().replace("\u200e",""))
                product_data.update(dict(zip(product_details_key,product_details_value)))
            except:
                pass
            pass
        return product_data
    else:
        product_data = {}
        product_data["error"]=response.status_code
        return product_data

In [ ]:
for i in valid_data:
    url=(i["url"])
    print(i["url"])
    temp_data=get_data(url)
    print(temp_data)
    i.update(temp_data)

In [ ]:
valid_data

[{'id': 2,
  'Asin': '000004458X',
  'country': 'de',
  'url': 'https://www.amazon.de/dp/000004458X',
  'S.No.': 3,
  'Product Title': 'Old Spice Shaving Cream - 70 g (original) - Pack of 2',
  'Product Image URL': 'https://images-eu.ssl-images-amazon.com/images/I/41xHbjdSXJL._SY300_SX300_QL70_ML2_.jpg',
  ' Item Package Dimensions L x W x H ': '18.3 x 6.7 x 3.9 centimetres',
  ' Package Weight ': '0.18 Kilograms',
  ' Item Weight ': '68 Grams',
  ' Manufacturer ': 'Old Spice',
  ' Brand Name ': 'Old Spice',
  ' Item Form ': 'Creme,Schaum',
  ' Scent ': 'Rasur',
  ' Item model number ': 'SG_000004458X_US',
  ' Package Dimensions ': '18.29 x 6.71 x 3.91 cm; 68.04 Grams',
  ' ASIN ': '000004458X'},
 {'id': 2424797,
  'Asin': '000004458X',
  'country': 'fr',
  'url': 'https://www.amazon.fr/dp/000004458X',
  'S.No.': 4,
  'error': 404},
 {'id': 7,
  'Asin': '000101742X',
  'country': 'de',
  'url': 'https://www.amazon.de/dp/000101742X',
  'S.No.': 17,
  'Product Title': 'Short Story: Violo

In [ ]:
df=pd.DataFrame(valid_data)
df

,id,Asin,country,url,S.No.,Product Title,Product Image URL,Item Package Dimensions L x W x H,Package Weight,Item Weight,...,Opiniones de los clientes,Partition,Dimensioni,Paperback,Broché,Copertina flessibile,Herausgeber,Sprache,Musiknoten,Pamphlet
0,2,000004458X,de,https://www.amazon.de/dp/000004458X,3,Old Spice Shaving Cream - 70 g (original) - Pa...,https://images-eu.ssl-images-amazon.com/images...,18.3 x 6.7 x 3.9 centimetres,0.18 Kilograms,68 Grams,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2424797,000004458X,fr,https://www.amazon.fr/dp/000004458X,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,000101742X,de,https://www.amazon.de/dp/000101742X,17,Short Story: Violoncello und Klavier. (Cello-B...,https://images-eu.ssl-images-amazon.com/images...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2424804,000101742X,fr,https://www.amazon.fr/dp/000101742X,18,Short story (dushkin/hegyi) violon Partitio...,https://images-eu.ssl-images-amazon.com/images...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,000102163X,de,https://www.amazon.de/dp/000102163X,21,Concerto grosso g-Moll: Weihnachtskonzert. op....,https://images-eu.ssl-images-amazon.com/images...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2425213,000417870X,fr,https://www.amazon.fr/dp/000417870X,848,Stucke(3) op.142 orgue Broché – 1 janvier 2000,https://images-eu.ssl-images-amazon.com/images...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,24 pages,NaN,NaN,NaN,NaN,NaN
90,405,000420736X,de,https://www.amazon.de/dp/000420736X,895,L'ARLESIENNE-SUITE NR. 2 ORCHESTRE Paperbac...,https://images-eu.ssl-images-amazon.com/images...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,2425236,000420736X,fr,https://www.amazon.fr/dp/000420736X,896,L'arlesienne-suite nr. 2 orchestre Broché –...,https://images-eu.ssl-images-amazon.com/images...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,413,000421143X,de,https://www.amazon.de/dp/000421143X,911,EIN DEUTSCHES REQUIEM 2 KLAV Paperback – 1 ...,https://images-eu.ssl-images-amazon.com/images...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df.to_excel("Final_output.xlsx")
df.to_json("Final_output.json")